In [ ]:
!pip install torch==2.7.0
!pip install numpy==2.2.6
!pip install matplotlib==3.10.3
!pip install pennylane==0.41.4
!pip install pennylane-lightning[gpu]==0.41.1

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import torch
import numpy as np
import pennylane as qml
import torch.nn as nn

##Implementation

In [ ]:
class sqRBM:
    """
    Semi-quantum Restricted Boltzmann Machine (sqRBM) using a quantum circuit for the hidden layer.

    Args:
        num_visible (int): Number of visible units.
        num_hidden (int): Number of hidden units (equal to number of qubits).
        lr (float): Learning rate for Adam optimizer.
        k (int): Number of Gibbs sampling steps.
        epochs (int): Total training epochs.
        gamma (float): Learning rate decay factor.
        sigma (float): Std. dev. of Gaussian noise added to visible units during sampling.
        batch_size (int): Batch size for training.
        weight_decay (float): Weight decay (L2 regularization).
    """

    def __init__(
        self, num_visible, num_hidden,
        lr=1e-3, k=10,
        epochs=100, gamma=0.99,
        sigma=1e-4, batch_size=300,
        weight_decay=1e-4
    ):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.lr = lr
        self.k = k
        self.epochs = epochs
        self.gamma = gamma
        self.sigma = sigma
        self.batch_size = batch_size

        self.usedEpochs = []
        self.rec_errors = []

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Using device:", self.device)

        # Initialize weights and biases
        std = np.sqrt(2.0 / (num_visible + num_hidden))
        self.w = torch.randn(num_visible, num_hidden, device=self.device) * std
        self.b = torch.zeros(num_visible, device=self.device)
        self.c = torch.zeros(num_hidden, device=self.device)

        # Persistent chain for CD-k
        self.fantasy = torch.randn(batch_size, num_visible, device=self.device)

        # Try using GPU-accelerated QPU backend, fallback to CPU
        try:
            self.dev = qml.device("lightning.gpu", wires=num_hidden)
        except:
            self.dev = qml.device("default.qubit", wires=num_hidden)
            print("Falling back to default.qubit (CPU)")

        # Define QNode for quantum expectation over hidden units
        @qml.qnode(self.dev, interface="torch", diff_method="adjoint")
        def circuit(v, w, c):
            angles = torch.matmul(v, w) + c
            angles = 2.0 * angles  # Optional scaling
            for _ in range(2):  # Circuit depth = 2
                for i in range(num_hidden):
                    qml.RY(angles[i], wires=i)
                    qml.RZ(angles[i], wires=i)
                for i in range(num_hidden - 1):
                    qml.CNOT(wires=[i, i + 1])
                if num_hidden > 1:
                    qml.CNOT(wires=[num_hidden - 1, 0])
            return [qml.expval(qml.PauliZ(i)) for i in range(num_hidden)]

        self.circuit = circuit

        # Optimizer and LR scheduler
        self.opt = torch.optim.Adam([self.w, self.b, self.c], lr=lr, weight_decay=weight_decay)
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.opt, gamma=gamma)

    def forward(self, v_batch: torch.Tensor) -> torch.Tensor:
        """
        Apply quantum circuit to each data sample in a batch to produce hidden probabilities.

        Args:
            v_batch (torch.Tensor): Visible batch [batch_size, num_visible].

        Returns:
            torch.Tensor: Hidden probabilities [batch_size, num_hidden].
        """
        v_batch = v_batch.to(self.device).float()
        tapes = []

        for v in v_batch:
            v = v.detach().cpu()
            w = self.w.detach().cpu()
            c = self.c.detach().cpu()
            with qml.tape.QuantumTape() as tape:
                angles = torch.matmul(v, w) + c
                angles = 2.0 * angles
                for _ in range(2):
                    for i in range(self.num_hidden):
                        qml.RY(angles[i], wires=i)
                        qml.RZ(angles[i], wires=i)
                    for i in range(self.num_hidden - 1):
                        qml.CNOT(wires=[i, i + 1])
                    if self.num_hidden > 1:
                        qml.CNOT(wires=[self.num_hidden - 1, 0])
                for i in range(self.num_hidden):
                    qml.expval(qml.PauliZ(i))
            tapes.append(tape)

        results = qml.execute(tapes, device=self.dev, interface="torch")
        results = [torch.tensor(r, dtype=torch.float32) for r in results]
        return torch.sigmoid(torch.stack(results).to(self.device))

    def sample_visible(self, h_prob: torch.Tensor) -> torch.Tensor:
        """
        Sample visible layer given hidden probabilities.

        Args:
            h_prob (torch.Tensor): Hidden activations [batch_size, num_hidden].

        Returns:
            torch.Tensor: Sampled visible units.
        """
        h_prob = h_prob.to(self.device)
        mean = torch.matmul(h_prob, self.w.T) + self.b
        return mean + self.sigma * torch.randn_like(mean)

    def fit(self, data: torch.Tensor, epochs: int = None):
        """
        Train the model on a dataset using contrastive divergence.

        Args:
            data (torch.Tensor): Input data [N, num_visible].
            epochs (int): Number of training epochs.
        """
        if epochs is None:
            epochs = self.epochs

        data = data.clone().detach().float().to(self.device)
        n = data.shape[0]

        best_error = float('inf')
        best_w = self.w.clone()
        best_b = self.b.clone()
        best_c = self.c.clone()

        for epoch in range(epochs):
            perm = torch.randperm(n, device=self.device)
            data_shuf = data[perm][:900]  # Truncate to fixed-size sample
            epoch_err = 0.0

            for i in range(0, len(data_shuf), self.batch_size):
                v0 = data_shuf[i : i + self.batch_size]
                if v0.size(0) < self.batch_size:
                    break

                h0 = self.forward(v0)
                hk = h0
                for _ in range(self.k):
                    vk = self.sample_visible(hk)
                    hk = self.forward(vk)

                # Positive and negative phase
                pos = torch.matmul(v0.T, h0)
                neg = torch.matmul(vk.T, hk)

                # Compute gradients (manual autograd)
                self.opt.zero_grad()
                self.w.grad = -(pos - neg - 0.001 * self.w)
                self.b.grad = -torch.sum(v0 - vk, dim=0)
                self.c.grad = -torch.sum(h0 - hk, dim=0)
                self.opt.step()

                self.scheduler.step()
                epoch_err += torch.sum((v0 - vk) ** 2).item() / 900

            # Early stopping and weight rollback if error increases
            if epoch_err <= best_error:
                best_error = epoch_err
                best_w = self.w.clone()
                best_b = self.b.clone()
                best_c = self.c.clone()
            else:
                with torch.no_grad():
                    self.w.copy_(best_w)
                    self.b.copy_(best_b)
                    self.c.copy_(best_c)

            self.rec_errors.append(best_error)
            self.usedEpochs.append(epoch)
            print(f"Epoch {epoch}/{epochs} — Error: {epoch_err:.6f} (Best: {best_error:.6f})")

    def recPlot(self):
        """
        Plot reconstruction error across epochs on a log scale.
        """
        import matplotlib.pyplot as plt
        plt.plot(self.usedEpochs, self.rec_errors, marker='o')
        plt.yscale('log')
        plt.xlabel('Epoch')
        plt.ylabel('L2 Reconstruction Error')
        plt.title('Reconstruction Error Over Time')
        plt.grid(True)
        plt.show()

##Example Initialisation and Training

In [ ]:
"""
sqRBM5H = sqRBM(30, 5)
training_data_pytorch = training_data_pytorch.cpu()
sqRBM5H.fit(training_data_pytorch)
"""